## Step 1: Install Dependencies


In [ ]:
!pip3 install ir-datasets 'python-terrier==0.10.0' 'git+https://github.com/tira-io/tira.git@pyterrier-artifacts#egg=tira&subdirectory=python-client' 'git+https://github.com/webis-de/auto-ir-metadata@dev' 'git+https://github.com/mam10eks/autoqrels.git' 'git+https://github.com/OpenWebSearch/wows-code.git#egg=wows-eval&subdirectory=ecir25/wows-eval' --break-system-packages

In [7]:
from tira.rest_api_client import Client

tira = Client()

In [2]:
from wows_eval import evaluate as wows_evaluate

from autoqrels.oneshot import DuoPrompt
import pandas as pd
pd.set_option('display.max_colwidth', None)

WOWS_DATASET_ID = 'wows-eval/pairwise-smoke-test-20250210-training'


In [4]:
BACKBONE_MODEL = "flan-t5-small"

PROMPT = """Determine if passage B is as relevant as passage A
for the given query.
'Passage A: "...{{ rel_doc_text | replace("\\"", "\'") }}..."
'Passage B: "...{{ unk_doc_text | replace("\\"", "\'") }}..."
'Query: "{{ query_text }}" '
"Is passage B as relevant as passage A? </s>"""

In [5]:
flan_t5_small_assessor = DuoPrompt(
    backbone=f'google/{BACKBONE_MODEL}',
    prompt=PROMPT,
    dataset=None
)

In [8]:
input_data = tira.pd.inputs(WOWS_DATASET_ID)
input_data.head(3)

,id,query,relevant,unknown
0,3d080873-98a1-4388-af86-fe2c8b47ebca,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"Randy Newman (album) Randy Newman is the debut recording by Randy Newman, released in 1968. Unlike his later albums which featured Newman and his piano backed by guitar, bass guitar and drums, Randy Newman was highly orchestral and aimed to blend the orchestra with Newman's voice and piano."
1,468a9e92-467f-47c9-810b-fe6fa9dca634,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"One of Monk's most important contributions to jazz was his use of space and simplicity in his performances and in his compositions. Monk's unconventional use of harmony and rhythm has had a lasting influence on jazz as well. Finally, Monk's compositions are very well-known, both by performers and by listeners."
2,846a69d0-0c0e-4d86-baf2-c3e8d31fdc86,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"Singing elegant, melancholic songs in a glamorously tattered voice, Leonard Cohen emerged from Montreal in the 1960s, an artist well into his thirties before he even made his first album. After a few records, he was royalty, on equal footing with Joni Mitchell, Randy Newman, and other top-notch singer-songwriters."


In [22]:
from tqdm import tqdm

def _group_per_query_and_relevant(df):
    inps = {}

    for _, i in df.iterrows():
        if i['query'] not in inps:
            inps[i['query']] = {}
        if i['relevant'] not in inps[i['query']]:
            inps[i['query']][i['relevant']] = {}
        
        inps[i['query']][i['relevant']][i['id']] = i['unknown']
    return inps

def _batches(df):
    grouped = _group_per_query_and_relevant(df)
    ret = []
    for query in grouped.keys():
        for relevant in grouped[query]:
            tmp = {'query': query, 'relevant': relevant, 'dids': [], 'texts': []}
            for did, text in grouped[query][relevant].items():
                tmp['dids'].append(did)
                tmp['texts'].append(text)
            ret.append(tmp)
    return ret

def predict(self, df):
    ret = {}
    df = df.copy()
    for i in tqdm(_batches(df)):
        preds = self.infer_oneshot_text(i['query'], i['relevant'], i['texts'])
        assert len(preds) == len(i['dids'])
        for id, pred in zip(i['dids'], preds):
            assert id not in ret
            ret[id] = pred
    df['probability_relevant'] = df['id'].apply(lambda i: ret[i])

    return df

In [24]:
foo = predict(flan_t5_small_assessor, input_data)

100%|██████████| 6/6 [00:06<00:00,  1.07s/it]


In [25]:
foo

,id,query,relevant,unknown,probability_relevant
0,3d080873-98a1-4388-af86-fe2c8b47ebca,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"Randy Newman (album) Randy Newman is the debut recording by Randy Newman, released in 1968. Unlike his later albums which featured Newman and his piano backed by guitar, bass guitar and drums, Randy Newman was highly orchestral and aimed to blend the orchestra with Newman's voice and piano.",0.616158
1,468a9e92-467f-47c9-810b-fe6fa9dca634,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"One of Monk's most important contributions to jazz was his use of space and simplicity in his performances and in his compositions. Monk's unconventional use of harmony and rhythm has had a lasting influence on jazz as well. Finally, Monk's compositions are very well-known, both by performers and by listeners.",0.713638
2,846a69d0-0c0e-4d86-baf2-c3e8d31fdc86,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"Singing elegant, melancholic songs in a glamorously tattered voice, Leonard Cohen emerged from Montreal in the 1960s, an artist well into his thirties before he even made his first album. After a few records, he was royalty, on equal footing with Joni Mitchell, Randy Newman, and other top-notch singer-songwriters.",0.684874
3,83c0e22c-b00f-4570-b1cb-027199c673d4,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"Walker, Texas Ranger. Chuck Norris thought “Eyes of a Ranger” would be the perfect theme song for his new show Walker, Texas Ranger. He wanted his friend Randy Travis should sing it, but CBS had a different idea: The network suggested Norris sing the theme himself.",0.641787
4,a88a0a31-4795-4c59-830b-848de52a7fd6,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,This is a reference to the minor controversy that brewed among Monk fans over the introduction of the new theme song It's A Jungle Out There written and performed by Randy Newman in the second season of Monk.,0.641475
5,19f3efb5-e3ba-460a-b630-9ed3c20ada6f,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"What is the music in chapter 33 of House of Cards? This is the episode in season 3 with the monks. I'm looking in particular for the music played when Frank is looking at the statues of Elanor and Rosevelt, about halfway through the episode. Also for the music at the end credits, which I suspect is by the same artist.",0.640281
6,50131d5a-6415-43bd-8850-a41fcabf54e2,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"However, as Brave 's soundtrack reveals, the movie is also noteworthy for being one of the studio's most musical films, especially for one not featuring music by Pixar's go-to songwriter Randy Newman.",0.614744
7,bbf50ee5-9021-494f-8a4c-14e3355d989a,who sings monk theme song,"It's a Jungle Out There is a song written by Randy Newman, and used as the theme song for the TV series Monk starting in its second season. In 2004, it won an Emmy Award for Outstanding Main Title Theme Music.","Randy Newman (album) Randy Newman is the debut recording by Randy Newman, released in 1968. Unlike his later